In [1]:
! pip install keras-rl2
! pip install chess
! pip install python-chess

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# ls drive/MyDrive/Data/Chess

In [4]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,\
     Input,BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# import gym_chess

import chess
from sys import platform
import os
import chess.engine


2021-12-15 10:57:54.769508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 10:57:54.769534: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12608206137687737734
]


2021-12-15 10:57:56.324652: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-15 10:57:56.325300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-15 10:57:56.325313: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-15 10:57:56.325333: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LoG): /proc/driver/nvidia/version does not exist


In [6]:
# %%
os.system('chmod +x stockfish_14.1_linux_x64')
engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish_14.1_linux_x64")


In [7]:
# # %%
# if platform == "linux" or platform == "linux2":
#     os.system('chmod +x ../stockfish/stockfish_14.1_linux_x64')
#     engine = chess.engine.SimpleEngine.popen_uci(r"../stockfish/stockfish_14.1_linux_x64")
# elif platform == "win32":
#     engine = chess.engine.SimpleEngine.popen_uci(r"../stockfish/stockfish_14.1_win_32bit.exe")

In [8]:
def find_move(env):
    result = engine.play(env.env, chess.engine.Limit(time=0.05))
    return result.move

In [9]:
STATE_SHAPE = (65, )
NB_ACTIONS = 4096

In [10]:
class ChessEnv:
    '''
    state - obser: ndarray - (65,): [:65] is flatten from int_board; [65] is color of bot; 1 is white and -1 is black
    step: int. step_range = (0, 4096) , is encoded from square A to square B (64 x 64 val)
    reward: int
    '''

    mapped = {
            'P': 10,     # White Pawn
            'p': -10,    # Black Pawn
            'N': 20,     # White Knight
            'n': -20,    # Black Knight
            'B': 30,     # White Bishop
            'b': -30,    # Black Bishop
            'R': 40,     # White Rook
            'r': -40,    # Black Rook
            'Q': 50,     # White Queen
            'q': -50,    # Black Queen
            'K': 900,     # White King
            'k': -900     # Black King
    }
    # state_shape = (8, 8)
    # nb_actions = 4096
    model = None
    
    def __init__(self, model: Sequential, neg_r_each_step = -1) -> None:
        self.env = chess.Board()
        self.model = model
        self.state = self.reset()
        # [-1] = 1 -> white, -1 -> black
        self.bot_color = self.env.turn * 2 - 1
        self.neg_r_each_step = neg_r_each_step

    def is_draw(self):
        if self.env.is_stalemate():
            print("statlemate")
            return True
        if self.env.is_fivefold_repetition():
            print("fivefold repetition")
            return True
        if self.env.is_seventyfive_moves():
            print("75 moves")
            return True
        if self.env.is_insufficient_material():
            print("Insufficient Material")
            return True
        return False

    def is_checkmate(self):
        # If There is checkmate then it will be TRUE else FALSE.It will be a boolean value.
        return self.env.is_checkmate()

    def convert_board_to_int(self):
        epd_string = self.env.epd()
        list_int = np.empty((0, ))
        for i in epd_string:
            if i == " ":
                list_int = list_int.reshape((8, 8))
                return list_int
            elif i != "/":
                if i in self.mapped:
                    list_int = np.append(list_int, self.mapped[i])
                else:
                    for counter in range(0, int(i)):
                        list_int = np.append(list_int, 0)
        list_int = list_int.reshape((8, 8))
        return list_int

    def get_state(self) -> np.ndarray:
        return np.append(self.convert_board_to_int().reshape(64,), self.env.turn * 2 - 1)

    def legal_moves(self):
        return list(self.env.legal_moves)

    def encodeMove(self, move_uci:str):
        if len(move_uci) != 4:
            raise ValueError()
        a, b = chess.parse_square(move_uci[:2]), chess.parse_square(move_uci[2:])
        return a * 64 + b

    def decodeMove(self, move_int:int):
        a, b = move_int//64, move_int%64
        # a, b = chess.square_name(a), chess.square_name(b)

        move = self.env.find_move(from_square= a,to_square= b)
        return move

    def render(self):
        print(self.env.unicode())

    def reset(self):
        # random state
        redo = True
        num_sample_steps = 0
        while redo:
            redo = False
            self.env = chess.Board()
            num_sample_steps = np.random.randint(0, 50)
            for i in range (num_sample_steps):
                lg_move = self.legal_moves()
                if len(lg_move) != 0:
                    move = np.random.choice(self.legal_moves())
                    self.env.push(move)
                else:
                    redo = True
                    break

        Q_val = self.model.predict(self.get_state().reshape((1, 1) + STATE_SHAPE)).reshape(-1, )
        print('Val:', min(Q_val), max(Q_val))
        return self.get_state()

    def step(self, action: int):
        reward = 0
        done = True
        
        try:
            # move in legal move
            move = self.decodeMove(action)

            # neg reward each step
            reward = self.neg_r_each_step

            # location to_square
            to_r, to_c = move.to_square//8, move.to_square%8
            reward -= self.state[(7 - to_r)*8 + to_c ] * self.bot_color

            # action
            self.env.push(move)
            self.state = self.get_state()

            # check end game
            if self.is_checkmate():
                reward += self.mapped['K']
                done = True
                print('Win')
            elif self.is_draw():
                reward += 300
                done = True

            # opponent's turn   
            else:
                done = False

                move = find_move(self)

                # location to_square
                to_r, to_c = move.to_square//8, move.to_square%8
                reward -= self.state[(7 - to_r)*8 + to_c ] * self.bot_color

                # action
                self.env.push(move)
                self.state = self.get_state()

                # check end game
                if self.is_checkmate():
                    reward -= self.mapped['K']
                    done = True
                    print("Lose")
                elif self.is_draw():
                    reward += 300
                    done = True

        except:
            # wrong move
            reward = -5000
            done = True
            print('wrong_move')

        return self.state, reward, done, {}


In [11]:
# model
model = Sequential()
model.add(Input((1, ) + STATE_SHAPE))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(NB_ACTIONS))
model.add(Activation('linear'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 65)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8448      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________________________________________________

In [12]:
env = ChessEnv(model, neg_r_each_step=-1)

Val: -51.18316 55.089497


/home/kienanh/.local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2021-12-15 10:57:56.915777: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1999965000 Hz


In [13]:
model.load_weights('chess_model_opt_init.h5')

In [14]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
for i in range (10):
  policy = EpsGreedyQPolicy(0.01)
  dqn = DQNAgent(model=model, nb_actions=NB_ACTIONS, memory=memory,
                target_model_update=1e-2, policy=policy)
  dqn.compile(Adam(lr=1e-4), metrics=['mae'])

  # Okay, now it's time to learn something! We visualize the training here for show, but this
  # slows down training quite a lot. You can always safely abort the training prematurely using
  # Ctrl + C.
  his = dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)
  
  model.save('chess_model_opt_init.h5')

/home/kienanh/.local/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Training for 50000 steps ...
Val: -42.17766 12371.252
wrong_move
     1/50000: episode: 1, duration: 0.196s, episode steps:   1, steps per second:   5, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2832.000 [2832.000, 2832.000],  loss: --, mae: --, mean_q: --
Val: -42.908497 12078.172
wrong_move
     2/50000: episode: 2, duration: 0.013s, episode steps:   1, steps per second:  80, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 3164.000 [3164.000, 3164.000],  loss: --, mae: --, mean_q: --
Val: -45.2683 12000.288
wrong_move
     3/50000: episode: 3, duration: 0.018s, episode steps:   1, steps per second:  55, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 480.000 [480.000, 480.000],  loss: --, mae: --, mean_q: --
Val: -41.717342 11960.793
wrong_move
     4/50000: episode: 4, duration: 0.009s, episode steps:   1, steps per second: 108, episode reward: -5000.000, mean re

/home/kienanh/.local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  1001/50000: episode: 999, duration: 2.146s, episode steps:   1, steps per second:   0, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 2832.000 [2832.000, 2832.000],  loss: --, mae: --, mean_q: --
Val: -41.61587 12002.155
wrong_move
  1002/50000: episode: 1000, duration: 0.062s, episode steps:   1, steps per second:  16, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 1786.000 [1786.000, 1786.000],  loss: 134609984.000000, mae: 9731.386719, mean_q: 12099.132812
Val: -41.05755 12025.95
wrong_move
  1003/50000: episode: 1001, duration: 0.064s, episode steps:   1, steps per second:  16, episode reward: -5000.000, mean reward: -5000.000 [-5000.000, -5000.000], mean action: 480.000 [480.000, 480.000],  loss: 130477632.000000, mae: 9731.349609, mean_q: 12094.203125
Val: -43.29867 12148.335
wrong_move
  1004/50000: episode: 1002, duration: 0.066s, episode steps:   1, steps per second:  15, episode reward: -5000.00